> **Hello everyone, my R skills are better than python.If there is any room for improvement, I hope you can give me more advice, thank you! ! !**

In [ ]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import json

Dataset Link here: https://www.kaggle.com/robikscube/ubiquant-parquet

From:https://www.kaggle.com/robikscube/fast-data-loading-and-low-mem-with-parquet-files

In [ ]:
%%time
train = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')

-----------------------------------------------------------------------------------------------------------------------------------------------------------------

## 1 Basic information

In [ ]:
train.info()

In [ ]:
train.head()

**Field information**
* row_id - A unique identifier for the row.
* time_id - The ID code for the time the data was gathered. The time IDs are in order, but the real time between the time IDs is not constant and will likely be shorter for the final private test set than in the training set.
* investment_id - The ID code for an investment. Not all investment have data in all time IDs.
* target - The target.
* [f_0:f_299] - Anonymized features generated from market data.

## 2 Discuss *row_id*，*time_id* and *investment_id*

**Question 1:**

1. Whether **row_id** can uniquely identify row data? 
2. Whether the structure of this field is spliced by **time_id** and **investment_id**？

In [ ]:
len(train['row_id'].unique())

In [ ]:
def paste0(range1, string1, range2):
    texts = [str(num1) + string1 + str(num2) for num1, num2 in zip(range1, range2)]

    return texts


texts = paste0(train["time_id"], "_", train['investment_id'])

In [ ]:
sum(texts==train["row_id"])

**Answer:**

1. yes
2. yes

**Question 2:**

1. How many types of investments are there?
1. Are the sample sizes for each investment similar?
1. Is the sample data corresponding to each investment product continuous in time? 
1. Are the investment_id numbers consecutive?


In [ ]:
train["investment_id"].nunique()

In [ ]:
max=train["investment_id"].max()
min=train["investment_id"].min()

In [ ]:
print(f"investment_id numbers range from {min} to {max}")

In [ ]:
temp_df=train['investment_id'].value_counts().rename_axis('investment_id').reset_index(name='counts')

In [ ]:
temp_df["investment_id"]=temp_df["investment_id"].apply(str)
temp_df= temp_df.sort_values('counts',ascending=False)
temp_df.head()

In [ ]:
plt.figure(figsize=(20,10))

ax1 = train['investment_id'].value_counts().sort_values(ascending=False).plot(
    kind='bar')
ax1.set_xlabel("Investment_id")
ax1.set_ylabel("Frequency")

plt.show()

In [ ]:
len(temp_df["investment_id"][temp_df["counts"]<=100])

In [ ]:
f1=train["time_id"][train["investment_id"]==1].diff().value_counts().rename_axis('time_interval').reset_index(name='freq')
f2=train["time_id"][train["investment_id"]==50].diff().value_counts().rename_axis('time_interval').reset_index(name='freq')
f3=train["time_id"][train["investment_id"]==77].diff().value_counts().rename_axis('time_interval').reset_index(name='freq')

In [ ]:

plt.figure(figsize= (15, 10))

plt.subplot(3,1,1)
f1["time_interval"]=f1["time_interval"].apply(str)
f1= f1.sort_values('freq',ascending=False)
plt.bar('time_interval', 'freq',data=f1)
#plt.xlabel("Time_interval", size=12)
plt.ylabel("Freq", size=15)
plt.title("investment=1")

plt.subplot(3,1,2)
f2["time_interval"]=f2["time_interval"].apply(str)
f2= f2.sort_values('freq',ascending=False)
plt.bar('time_interval', 'freq',data=f2)
#plt.xlabel("Time_interval", size=12)
plt.ylabel("Freq", size=15)
plt.title("investment=50")
#plt.figure(figsize= (20, 10))

plt.subplot(3,1,3)
f3["time_interval"]=f3["time_interval"].apply(str)
f3= f3.sort_values('freq',ascending=False)
plt.bar('time_interval', 'freq',data=f3)
plt.xlabel("Time_interval", size=12)
plt.ylabel("Freq", size=15)
plt.title("investment=77")
#plt.figure(figsize= (20, 10))

plt.show()

In [ ]:
train.head(1)

In [ ]:
train.investment_id.nunique()

In [ ]:
train.investment_id.max()

**Answer:**

1. The number of investment types is 3579, however, the maximum value of investment_id exceeds this value, this result means that the asset number is not consecutive;
1. The sample size of each investment product is unbalanced, and there are 46 investment products with a sample size of less than 100;
1. The time series corresponding to investment products are not completely continuous 
1. No. The maximum value of investment_id is not equal to the number of unique values in this field.

**Question 3:**

1. Are the time_id numbers consecutive?

In [ ]:
train.time_id.nunique()

In [ ]:
train.time_id.max()

**Answer:**

1. No. The maximum value of time_id is not equal to the number of unique values in this field.

## 3 Features & Target

In [ ]:
plt.figure(figsize= (15, 10))
plt.hist('target',1000,data=train)
plt.xlabel("Target", size=12)
plt.ylabel("Freq", size=15)
plt.show()

In [ ]:
plt.figure(figsize= (10, 40))

plt.subplot(10,1,1)
plt.hist('f_0',1000,data=train)
#plt.xlabel("Target", size=12)
plt.ylabel("Freq", size=15)
plt.title("Feature:f_0")

plt.subplot(10,1,2)
plt.hist('f_1',1000,data=train)
#plt.xlabel("Target", size=12)
plt.ylabel("Freq", size=15)
plt.title("Feature:f_1")

plt.subplot(10,1,3)
plt.hist('f_2',1000,data=train)
plt.xlabel("Target", size=12)
plt.ylabel("Freq", size=15)
plt.title("Feature:f_2")

plt.subplot(10,1,4)
plt.hist('f_3',1000,data=train)
plt.xlabel("Target", size=12)
plt.ylabel("Freq", size=15)
plt.title("Feature:f_3")

plt.subplot(10,1,5)
plt.hist('f_4',1000,data=train)
plt.xlabel("Target", size=12)
plt.ylabel("Freq", size=15)
plt.title("Feature:f_4")

plt.subplot(10,1,6)
plt.hist('f_5',1000,data=train)
plt.xlabel("Target", size=12)
plt.ylabel("Freq", size=15)
plt.title("Feature:f_5")

plt.subplot(10,1,7)
plt.hist('f_6',1000,data=train)
plt.xlabel("Target", size=12)
plt.ylabel("Freq", size=15)
plt.title("Feature:f_6")

plt.subplot(10,1,8)
plt.hist('f_7',1000,data=train)
plt.xlabel("Target", size=12)
plt.ylabel("Freq", size=15)
plt.title("Feature:f_7")

plt.subplot(10,1,9)
plt.hist('f_8',1000,data=train)
plt.xlabel("Target", size=12)
plt.ylabel("Freq", size=15)
plt.title("Feature:f_8")

plt.subplot(10,1,10)
plt.hist('f_9',1000,data=train)
plt.xlabel("Target", size=12)
plt.ylabel("Freq", size=15)
plt.title("Feature:f_9")

plt.show()

**To be continued. Please stay tuned!**